In [24]:
# ----------------------------------------------- AUXILIARY OPERATIONS ----------------------------------------------- 

def validate_params(N, p, q, d):
    """ check current ntru parameters
    """
    assert N.is_prime(), "N is not a prime"
    assert p.is_prime(), "p is not a prime"
    assert q.is_power_of(2), "q must be a power of two"
    assert q > p and gcd(p,q) == 1, "q must be larger than p and the greatest common divider of p and q is 1"
    assert (2*d + 1) <= N
    # assert q > (6*d + 1) * p, "q must be more then (6*d + 1) * p"
        
def find_degree(coefs_list):
    """ returns the degree of polynomial using its coeficients list 'a'
    """
    for i in range(len(coefs_list)-1, -1, -1):
        if coefs_list[i] != 0:
            return i

def invertmodprime(f, p, N):
    ''' calculates an inversion of a polynomial modulo x^N-1 and then modulo p
        with assumption that p is prime.
        returns a Zx polynomial h such as convolution of h ~ f = 1 (mod p)                
        raises an exception if such Zx polynomial h doesn't exist'''

    T = Zx.change_ring(Integers(p)).quotient(x^N-1) 
    return Zx(lift(1 / T(f)))                      

def invertmodpowerof2(a, p, N):
    """
    calculates an inversion of a polynomial modulo x^N-1 and then modulo p
    """
    r = int(math.log(p, 2))
    p = 2
    
    q = p
    b = invertmodprime(a, p, N)

    while q < p^r:
        q = q^2
        b = b * (2 - a*b) % q % (x^N-1)
        
    b = b % p^r % (x^N - 1)
    return b   

def balancedmod(f, q , N):
    ''' reduces every coefficient of a Zx polynomial f modulo q
        with additional balancing, so the result coefficients are integers in interval [-q/2, +q/2]
        more specifically: for an odd q [-(q-1)/2, +(q-1)/2], for an even q [-q/2, +q/2-1]. 
        returns Zx reduced polynomial'''

    g = list(((f[i] + q//2) % q) - q//2 for i in range(N))
    return Zx(g)

def convolution(f, g, N):
    ''' performs a multiplication operation specific for NTRU, which works like a traditional polynomial multiplication
        with additional reduction of the result by x^N-1 (x^n is replaced by 1, x^n-1 by x, x^n-2 by x^2, ...)
        returns Zx polynomial'''
    
    return (f * g) % (x^N-1)

# ----------------------------------------------- SETUP -----------------------------------------------

def generate_polynomial(d1, d2, N):
    ''' generates a random polynomial with d nonzero coefficients
        returns Zx polynomial '''
    
    result = [1]*d1 + [-1]*d2 + [0]*(N-d1-d2)
    shuffle(result)
    
    return Zx(result)

def generate_keys(N, p, q, d, p1 = None, p2= None):
    ''' generates a public and private key pair, based on provided parameters
        returns Zx public key and a secret key as a tuple of Zx f (private key) and Zx F_p'''

        #   some polynomials are not invertible and as f and g are calculated randomly,
        #   it may be necessary to skip some invalid examples
    while True:
        try:
            # generate 2 random polynomials f and g with number of nonzero coefficients < given number
            f = p1 or generate_polynomial(d + 1, d, N)
            g = p2 or generate_polynomial(d, d, N)

            # formula: find f_q, where: f_q (*) f = 1 (mod q)
            # assuming q is a power of 2                 
            f_q = invertmodpowerof2(f, q, N)

            # formula: find f_p, where: f_p (*) f = 1 (mod p) 
            # assuming p is a prime number 
            f_p = invertmodprime(f, p, N)
            break

        except:
            pass

    # formula: public key = F_q ~ g (mod q)
    public_key = balancedmod(p * convolution(f_q, g, N), q, N)

    # secret key is a tuple containing a private key (f) and variable f_p needed for decryption
    secret_key = f, f_p

    return public_key, secret_key
    
#---------------------------------- ENCRYPTION -----------------------------------------
def generate_message(N, p, q, d):
    ''' creates a polynomial from a random list of coefficients selected from a set {-1,0,1}  
        returns Zx polynomial'''
        
    #randrange(3) - 1 gives results from a set of {-1,0,1}, which is necessary for a proper decryption
    result = list(randrange(3) - 1 for j in range(N))
    return Zx(result)

def encrypt(message, public_key, N, p, q, d, r = None):
    ''' performs encryption of a given message using a provided public key
        returns Zx encrypted message'''

    # generate random polynomial with number of nonzero coefficients < N for adding extra noise  
    r = r or generate_polynomial(d, d-1, N)
    
    # formula: encrypted_message = p * r ~ public_key + message (mod q)
    # while performing modulo operation, balance coefficients of encrypted_message 
    # for the integers in interval [-q/2, +q/2]
    return balancedmod(convolution(public_key,r, N) + message,q, N)


def decrypt(encrypted_message, secret_key, N, p, q, d):
    ''' performs decryption of a given ciphertext using an own private key
        
        returns Zx decrypted message'''
    
    # private key - f; additional variable stored for decryption - f_p     
    f,f_p = secret_key
    
    # formula: a = f ~ encrypted_message (mod q)
    # balance coefficients of a for the integers in interval [-q/2, +q/2]
    a = balancedmod(convolution(encrypted_message,f, N),q, N)
     
    # formula: F_p ~ a (mod p) with additional balancing as above
    return balancedmod(convolution(a,f_p, N),p, N)


In [25]:
from random import shuffle
from math import log2

Zx.<x> = ZZ[]

def ntru_session(N, p, q, d, message = None, f = None, g = None, r = None):
    
    validate_params(N, p, q, d)
    N, p, q, d = N, p, q, d
    
    print("N, p, q, d : ", N, p, q, d)
    
    if message is not None:
        assert len(message.coefficients(sparse=False)) <= N, "invalid message"
    if f is not None:
        assert len(f.coefficients(sparse=False)) <= N, "invalid f degree"
    if g is not None:
        assert len(g.coefficients(sparse=False)) <= N, "invalid g degree"
    if r is not None:
        assert len(r.coefficients(sparse=False)) <= N, "invalid r degree"
    
    message = message or generate_message(N, p, q, d)
    public_key, secret_key = generate_keys(N, p, q, d, f, g)
    print("MESSAGE: " + str(message))
    encrypted_message = encrypt(message, public_key, N, p, q, d, r)
    print("ENCRYPTION: " + str(encrypted_message))
    decrypted_message = decrypt(encrypted_message, secret_key, N, p, q, d)
    print("DECRYPTION: " + str(decrypted_message))
    
    if message == decrypted_message:
        return True
        print("SUCCESS")
    else:
        return False
        print("FAIL")
    
    

In [26]:
def test_wikipedia():
    print(f"TEST CASE: ntru data from wikipedia")
    try:
        f = -1+x+x^2-x^4+x^6+x^9-x^10
        g = -1+x^2+x^3+x^5-x^8-x^10
        r = -1+x^2+x^3+x^4-x^5-x^7
        message = -1+x^3-x^4-x^8+x^9+x^10
        res = ntru_session(11, 3, 32, 3, message, f, g, r)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
        
test_wikipedia()

TEST CASE: ntru data from wikipedia
N, p, q, d :  11 3 32 3
MESSAGE: x^10 + x^9 - x^8 - x^4 + x^3 - 1
ENCRYPTION: -13*x^10 + 6*x^9 - 7*x^8 + 7*x^7 - 2*x^6 - 16*x^5 + 14*x^4 - 8*x^3 - 6*x^2 + 11*x + 14
DECRYPTION: x^10 + x^9 - x^8 - x^4 + x^3 - 1
TEST PASSED
*******



In [27]:
def test_small_parameters_all_random():
    print(f"TEST CASE: test with small parameters and all are random")
    try:
        res = ntru_session(11, 3, 32, 3)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
        
test_small_parameters_all_random()

TEST CASE: test with small parameters and all are random
N, p, q, d :  11 3 32 3
MESSAGE: x^10 + x^6 - x^5 + x^4 - x^2 - 1
ENCRYPTION: -15*x^10 + 9*x^9 + 4*x^8 + 10*x^7 + 15*x^6 - 12*x^5 - 2*x^4 + 14*x^3 - 3*x^2 + 6*x + 6
DECRYPTION: x^10 + x^6 - x^5 + x^4 - x^2 - 1
TEST PASSED
*******



In [28]:
def test_large_parameters_all_random(N, p, q, d):
    print(f"TEST CASE: test with large parameters and all are random")
    try:
        res = ntru_session(N, p, q, d)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
        
test_large_parameters_all_random(701, 3, 8192, 103)

TEST CASE: test with large parameters and all are random
N, p, q, d :  701 3 8192 103
MESSAGE: x^700 + x^699 - x^698 - x^695 + x^693 - x^692 + x^691 - x^690 + x^689 + x^688 - x^687 + x^686 + x^684 - x^683 - x^682 - x^681 - x^680 - x^678 + x^677 - x^675 - x^674 + x^673 + x^672 + x^670 - x^669 + x^667 - x^666 + x^665 - x^664 - x^662 - x^660 + x^659 - x^658 - x^657 + x^655 - x^653 + x^652 + x^651 + x^650 + x^646 - x^645 - x^644 - x^643 - x^642 - x^641 - x^640 + x^636 - x^633 + x^630 - x^629 + x^627 - x^626 - x^625 - x^624 + x^620 - x^619 - x^618 - x^617 - x^614 + x^613 - x^610 + x^605 + x^604 + x^603 - x^601 - x^600 - x^599 - x^598 + x^595 - x^594 + x^593 - x^592 - x^591 - x^589 + x^588 + x^586 - x^583 - x^581 + x^579 - x^578 - x^577 - x^576 + x^574 - x^573 - x^572 + x^571 - x^570 + x^569 + x^568 - x^566 + x^565 - x^563 + x^562 + x^561 + x^558 + x^555 - x^554 + x^551 + x^550 + x^549 - x^548 + x^546 - x^545 - x^540 + x^537 - x^536 + x^534 + x^533 - x^531 + x^526 - x^522 + x^521 - x^519 - x

In [29]:
def test_lecture(N, p, q, d):
    print(f"TEST CASE: ntru data from lecture")
    try:
        res = ntru_session(N, p, q, d)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except AssertionError as e :
        print(e)
        print("TEST PASSED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
        
        
test_lecture(7, 3, 41, 2)

TEST CASE: ntru data from lecture
q must be a power of two
TEST PASSED
*******



In [30]:
def test_small_parameters_given_f(f):
    print(f"TEST CASE: test with small parameters and given polynomial f")
    try:
        res = ntru_session(11, 3, 32, 3, f=f)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
       
    

test_small_parameters_given_f(-1+x+x^2-x^4+x^6+x^9-x^10)

TEST CASE: test with small parameters and given polynomial f
N, p, q, d :  11 3 32 3
MESSAGE: -x^10 + x^6 + x^5 - x^4 + 1
ENCRYPTION: -9*x^10 - 9*x^9 - 13*x^8 + 2*x^7 - 6*x^6 - 6*x^5 - 11*x^3 + 4*x^2 - 2*x - 13
DECRYPTION: -x^10 + x^6 + x^5 - x^4 + 1
TEST PASSED
*******



In [31]:
def test_small_parameters_given_invalid_f(f):
    print(f"TEST CASE: test with small parameters and invalid given polynomial f")
    try:
        res = ntru_session(11, 3, 32, 3, f=f)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except AssertionError as e :
        print(e)
        print("TEST PASSED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
       
    
test_small_parameters_given_invalid_f(-1+x+x^2-x^4+x^6+x^9-x^12)

TEST CASE: test with small parameters and invalid given polynomial f
N, p, q, d :  11 3 32 3
invalid f degree
TEST PASSED
*******



In [32]:
def test_small_parameters_given_message(message):
    print(f"TEST CASE: test with small parameters and invalid given polynomial f")
    try:
        res = ntru_session(11, 3, 32, 3, message=message)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except AssertionError as e :
        print(e)
        print("TEST PASSED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
       
    
test_small_parameters_given_message(-1+x+x^2-x^4+x^6+x^9)

TEST CASE: test with small parameters and invalid given polynomial f
N, p, q, d :  11 3 32 3
MESSAGE: x^9 + x^6 - x^4 + x^2 + x - 1
ENCRYPTION: -5*x^10 - 7*x^9 + 4*x^8 - 2*x^7 - 14*x^6 - 6*x^5 - 4*x^4 - 15*x^3 + 2*x^2 - 9*x - 6
DECRYPTION: x^9 + x^6 - x^4 + x^2 + x - 1
TEST PASSED
*******



In [33]:
def test_small_parameters_given_invalid_message(message):
    print(f"TEST CASE: test with small parameters and invalid given polynomial f")
    try:
        res = ntru_session(11, 3, 32, 3, message=message)
        if res:
            print("TEST PASSED")
        else:
            print("TEST FAILED")
    except AssertionError as e :
        print(e)
        print("TEST PASSED")
    except:
        print("TEST FAILED")
    finally: 
        print("*******\n")
       
    
test_small_parameters_given_invalid_message(-1+x+x^2-x^4+x^6+x^23)

TEST CASE: test with small parameters and invalid given polynomial f
N, p, q, d :  11 3 32 3
invalid message
TEST PASSED
*******



In [34]:
from numpy import base_repr

def encode(s):
    lst = [ord(i) for i in s]
    lst2 = [base_repr(i,base=3) for i in lst]
    lst3 = []
    for i in lst2:
        lst3 += i
    res = [int(i) - 1 for i in lst3]
    m = Zx(res)
    return m

M = encode("my_message")

In [35]:
ntru_session(701, 3, 8192, 103, m)

N, p, q, d :  701 3 8192 103
MESSAGE: x^48 - x^47 + x^46 - x^45 + x^41 - x^40 + x^37 - x^35 + x^32 - x^31 + x^27 - x^26 + x^23 - x^22 + x^21 - x^20 - x^17 - x^16 + x^13 - x^11 - x^3 - x^2
ENCRYPTION: -3833*x^700 - 1119*x^699 + 1042*x^698 - 2106*x^697 - 461*x^696 - 847*x^695 - 1826*x^694 + 157*x^693 + 1129*x^692 + 1568*x^691 - 3681*x^690 - 1213*x^689 + 2164*x^688 - 1975*x^687 - 1480*x^686 - 2394*x^685 - 2213*x^684 + 3562*x^683 - 695*x^682 - 128*x^681 + 3540*x^680 - 2809*x^679 + 2048*x^678 - 862*x^677 + 1512*x^676 - 3520*x^675 + 3960*x^674 - 1996*x^673 - 2040*x^672 - 3049*x^671 + 571*x^670 + 16*x^669 - 88*x^668 - 3330*x^667 + 545*x^666 - 803*x^665 + 252*x^664 + 2731*x^663 - 3483*x^662 - 2680*x^661 - 3122*x^660 + 1711*x^659 - 2631*x^658 + 3819*x^657 - 3878*x^656 - 331*x^655 + 2127*x^654 - 1631*x^653 + 3494*x^652 + 2957*x^651 - 862*x^650 + 1207*x^649 - 837*x^648 - 2971*x^647 + 479*x^646 + 1020*x^645 - 368*x^644 - 97*x^643 + 2110*x^642 + 2392*x^641 - 2533*x^640 - 2639*x^639 - 1241*x^638 + 3

True